In [1]:
from collections import OrderedDict
import itertools
import math
import pandas as pd
import pulp as pl

In [3]:
# PROBLEM DATA

# Time axis: months when purchases can be made (and capacities persist).
# Use an OrderedDict to keep a stable, readable order in outputs.
MONTHS = list(OrderedDict.fromkeys(["Jan", "Feb", "Mar", "Apr"]))

# Departments that "go live" the month AFTER licenses must be in place.
# We encode requirements as "cumulative seats required by end of month m".
cum_required_by_end = {
    "Jan":  50,   # to have HR live in Feb
    "Feb": 250,   # HR + Warehouses live in Mar
    "Mar": 270,   # + Strategy live in Apr
    "Apr": 355,   # + Store Leads live in May
}

# License packs (seats) and base (non-discounted) costs
# NOTE: 'Elite' has a discount in Jan & Feb; handled below in get_cost()
LICENSES = OrderedDict({
    "Basic":  {"seats": 30,   "base_cost":  2500},
    "Pro":    {"seats": 80,   "base_cost":  5000},
    "Elite":  {"seats": 200,  "base_cost": 10000},  # 10% off in Jan & Feb
    "Global": {"seats": 2000, "base_cost": 25000},
})

# Elite discount: 10% off in Jan and Feb
ELITE_DISCOUNT_MONTHS = {"Jan", "Feb"}
ELITE_DISCOUNT_FACTOR  = 0.90  # 10% off

def get_cost(month: str, lic_type: str) -> int:
    """Return the month-specific cost for a license type."""
    base = LICENSES[lic_type]["base_cost"]
    if lic_type == "Elite" and month in ELITE_DISCOUNT_MONTHS:
        return int(base * ELITE_DISCOUNT_FACTOR)
    return base

# Warehouses require at least one Pro-or-higher license by the time they go live.
# Since Warehouses go live in March, we must have at least one Pro/Elite/Global
# purchased by END OF FEB.
WAREHOUSE_PRO_PLUS_DEADLINE = "Feb"
PRO_OR_HIGHER = {"Pro", "Elite", "Global"}

In [4]:
# BUILD IP MODEL

model = pl.LpProblem("RetailPower_full_horizon_IP", pl.LpMinimize)

# Decision variables: x[(m, t)] = integer # of licenses of type t purchased in month m.
x = pl.LpVariable.dicts(
    "x",
    [(m, t) for m in MONTHS for t in LICENSES.keys()],
    lowBound=0,
    cat="Integer",
)

# Objective: minimize total spend over all months & license types
model += pl.lpSum(
    get_cost(m, t) * x[(m, t)]
    for m in MONTHS
    for t in LICENSES.keys()
), "Total_Cost"

# Helper: cumulative seats available by end of month m
# S_m = sum_{m' <= m} sum_t seats_t * x[m', t]
def months_up_to(m_inclusive):
    """Return list of months up to and including m_inclusive, respecting MONTHS order."""
    idx = MONTHS.index(m_inclusive)
    return MONTHS[: idx + 1]

# Cumulative coverage constraints
for m in MONTHS:
    required = cum_required_by_end.get(m, 0)
    if required <= 0:
        continue  # no constraint if requirement is zero or not specified
    model += (
        pl.lpSum(LICENSES[t]["seats"] * x[(mp, t)]
                 for mp in months_up_to(m)
                 for t in LICENSES.keys())
        >= required,
        f"Cumulative_Seats_by_end_of_{m}_>=_{required}"
    )

# Warehouses Pro+ rule: by end of Feb, at least one Pro/Elite/Global has been purchased.
deadline = WAREHOUSE_PRO_PLUS_DEADLINE
model += (
    pl.lpSum(
        x[(mp, t)]
        for mp in months_up_to(deadline)
        for t in PRO_OR_HIGHER
    ) >= 1,
    f"Warehouse_ProPlus_by_end_of_{deadline}"
)

In [7]:
# SOLVE

solver = pl.PULP_CBC_CMD(msg=False) # changing msg=True shows solver output
status = model.solve(solver)

In [16]:
# DISPLAY SOLUTION

status_name = pl.LpStatus[model.status]
print(f"Solver status: {status_name}")

total_cost = pl.value(model.objective)
print(f"Optimal total cost: ${total_cost:,.0f}\n")

# Purchases by month × license type
data_rows = []
for m in MONTHS:
    for t in LICENSES.keys():
        qty = int(pl.value(x[(m, t)]))
        if qty != 0:
            data_rows.append({
                "Month": m,
                "License": t,
                "Qty": qty,
                "Seats per Lic": LICENSES[t]["seats"],
                "Cost per Lic": get_cost(m, t),
                "Subtotal Cost": qty * get_cost(m, t),
                "Seats Provided": qty * LICENSES[t]["seats"],
            })

# If no purchases recorded (shouldn't happen with these numbers), fill zeros
if not data_rows:
    for m in MONTHS:
        for t in LICENSES.keys():
            data_rows.append({
                "Month": m, "License": t, "Qty": 0,
                "Seats per Lic": LICENSES[t]["seats"],
                "Cost per Lic": get_cost(m, t),
                "Subtotal Cost": 0,
                "Seats Provided": 0,
            })

# Pretty print purchases
df_purchases = pd.DataFrame(data_rows) 
display(df_purchases)
    
# Cumulative seats by month + requirement checks
cum_rows = []
for m in MONTHS:
    seats_cum = 0
    for mp in months_up_to(m):
        for t in LICENSES.keys():
            seats_cum += LICENSES[t]["seats"] * int(pl.value(x[(mp, t)]))
    req = cum_required_by_end.get(m, 0)
    cum_rows.append({
        "Month": m,
        "Cumulative Seats": seats_cum,
        "Required Seats": req,
        "Extra Seats": seats_cum - req
    })

df_cum = pd.DataFrame(cum_rows)
display(df_cum)

# Grand totals (sanity check)
total_seats = sum(LICENSES[t]["seats"] * int(pl.value(x[(m, t)]))
                      for m in MONTHS for t in LICENSES.keys())
print(f"\nGrand total seats purchased (all months): {total_seats:,}")
print(f"Grand total cost (matches objective): ${total_cost:,.0f}")


Solver status: Optimal
Optimal total cost: $18,000



,Month,License,Qty,Seats per Lic,Cost per Lic,Subtotal Cost,Seats Provided
0,Jan,Elite,1,200,9000,9000,200
1,Feb,Elite,1,200,9000,9000,200


,Month,Cumulative Seats,Required Seats,Extra Seats
0,Jan,200,50,150
1,Feb,400,250,150
2,Mar,400,270,130
3,Apr,400,355,45



Grand total seats purchased (all months): 400
Grand total cost (matches objective): $18,000
